In [3]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

In [4]:
df = pd.read_csv("Divvy_Trips_2023_full.csv.gz", compression="gzip")

In [5]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,F96D5A74A3E41399,electric_bike,2023-01-21 20:05:42,2023-01-21 20:16:33,Lincoln Ave & Fullerton Ave,TA1309000058,Hampden Ct & Diversey Ave,202480.0,41.924074,-87.646278,41.930000,-87.640000,member
1,13CB7EB698CEDB88,classic_bike,2023-01-10 15:37:36,2023-01-10 15:46:05,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member
2,BD88A2E670661CE5,electric_bike,2023-01-02 07:51:57,2023-01-02 08:05:11,Western Ave & Lunt Ave,RP-005,Valli Produce - Evanston Plaza,599,42.008571,-87.690483,42.039742,-87.699413,casual
3,C90792D034FED968,classic_bike,2023-01-22 10:52:58,2023-01-22 11:01:44,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member
4,3397017529188E8A,classic_bike,2023-01-12 13:58:01,2023-01-12 14:13:20,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member


# Критерии нахождения станции с высокой маятниковостью
Будем считать что есть несколько критериев, которые позволяют определить станцию с высокой маятниковостью:
- **Большая разница между приехавшими/уехавшими велосипедами утром и вечером**: если разница между этими показателями большая то это может означать что велосипеды уезжают в другие районы города, а возвращаются только вечером. И наоборот - если разница между показателями маленькая, значит нет четкого распределения поездок в/с этой станции.
- **Большое количество коротких поездок**: обычно маятниковые поездки связаны с небольшими перемещениями (например, с работы домой и назад) по городу, а не с длинными прогулочными поездками. Так что можно сделать вывод - если в станции много коротких поездок, то это может быть признаком маятниковости.

Попробуем рассчитать эти показатели для каждой станции и найти станции с самой ярко выраженной маятниковостью.

In [6]:
# Считаем сколько поездок началось/закончилось в каждой станции
start_count = df.groupby("start_station_name")["ride_id"].count().reset_index()
start_count.columns = ["station_name", "start_count"]
end_count = df.groupby("end_station_name")["ride_id"].count().reset_index()
end_count.columns = ["station_name", "end_count"]
station_counts = pd.merge(start_count, end_count, on="station_name", how="outer")
station_counts = station_counts.fillna(0)
station_counts["total_count"] = (
    station_counts["start_count"] + station_counts["end_count"]
)


## Считаем сколько поездок началось/закончилось утром/вечером


In [7]:
# Получаем час начала и конца поездки
df["start_hour"] = pd.to_datetime(df["started_at"]).dt.hour
df["end_hour"] = pd.to_datetime(df["ended_at"]).dt.hour

# Разделяем утренние и вечерние поездки
morning_trips = df[(df["start_hour"] >= 6) & (df["start_hour"] < 12)]
evening_trips = df[(df["end_hour"] >= 15) & (df["end_hour"] < 20)]

# Получаем сколько приехало/уехало с каждой станции утром
morning_out_counts = (
    morning_trips.groupby("start_station_name")["ride_id"].count().reset_index()
)
morning_out_counts.columns = ["station_name", "morning_in_count"]
morning_in_counts = (
    morning_trips.groupby("end_station_name")["ride_id"].count().reset_index()
)
morning_in_counts.columns = ["station_name", "morning_out_count"]
morning_counts = pd.merge(
    morning_in_counts, morning_out_counts, on="station_name", how="outer"
)

# Получаем сколько приехало/уехало с каждой станции вечером
evening_in_counts = (
    evening_trips.groupby("end_station_name")["ride_id"].count().reset_index()
)
evening_in_counts.columns = ["station_name", "evening_in_count"]
evening_out_counts = (
    evening_trips.groupby("start_station_name")["ride_id"].count().reset_index()
)
evening_out_counts.columns = ["station_name", "evening_out_count"]
evening_counts = pd.merge(
    evening_in_counts, evening_out_counts, on="station_name", how="outer"
)

# Объединяем датасеты
station_counts = pd.merge(
    station_counts, morning_counts, on="station_name", how="outer"
)
station_counts = pd.merge(
    station_counts, evening_counts, on="station_name", how="outer"
)
station_counts = station_counts.fillna(0)


## Считаем длительность поездки и количество коротких поездок

In [8]:
# Продолжительность поездки
df["tripduration"] = (
    pd.to_datetime(df["ended_at"]) - pd.to_datetime(df["started_at"])
).dt.total_seconds()
short_trips = df[df["tripduration"] <= 30 * 60]
# Количество коротких поездок
short_counts = (
    short_trips.groupby("start_station_name")["ride_id"].count().reset_index()
)
short_counts.columns = ["station_name", "short_count"]
station_counts = pd.merge(station_counts, short_counts, on="station_name", how="outer")
station_counts = station_counts.fillna(0)

In [9]:
# Считаем долю коротких поездок
station_counts["short_ratio"] = (
    station_counts["short_count"] / station_counts["total_count"]
)

In [10]:
# Считаем общее количество поездок утром и вечером
station_counts["morning_count"] = (
    station_counts["morning_in_count"] + station_counts["morning_out_count"]
)
station_counts["evening_count"] = (
    station_counts["evening_in_count"] + station_counts["evening_out_count"]
)

# Считаем разницу между количеством поездок, начавшихся и закончившихся в станции утром и вечером
station_counts["morning_swing"] = abs(
    station_counts["morning_in_count"] - station_counts["morning_out_count"]
)
station_counts["evening_swing"] = abs(
    station_counts["evening_in_count"] - station_counts["evening_out_count"]
)
station_counts["swing"] = (
    station_counts["morning_swing"] + station_counts["evening_swing"]
)

## Находим станции с высокой маятниковостью
 - Много коротких поездок

Cравнение двух показателей утром и вечером:
- (Утром со станции больше уезжают чем приезжают) И (вечером на станцию больше приезжают чем уезжают)
- (Утром на станцию больше приезжают чем уезжают) И (вечером со станции больше уезжают чем приезжают)

In [11]:
swing_stations = station_counts[
    (station_counts["swing"] > 10)
    & (
        (
            (station_counts["morning_out_count"] > station_counts["morning_in_count"])
            & (station_counts["evening_out_count"] < station_counts["evening_in_count"])
        )
        | (
            (station_counts["morning_out_count"] < station_counts["morning_in_count"])
            & (station_counts["evening_out_count"] > station_counts["evening_in_count"])
        )
    )
    & (station_counts["short_ratio"] > 0.3)
]
swing_stations = swing_stations.sort_values(by=["swing"], ascending=False)

In [12]:
swing_stations.head(100)

,station_name,start_count,end_count,total_count,morning_out_count,morning_in_count,evening_in_count,evening_out_count,short_count,short_ratio,morning_count,evening_count,morning_swing,evening_swing,swing
182,Columbus Dr & Randolph St,2191.0,1781.0,3972.0,642.0,833.0,731.0,852.0,2044.0,0.514602,1475.0,1583.0,191.0,121.0,312.0
328,Halsted St & Clybourn Ave,2465.0,2586.0,5051.0,567.0,544.0,1334.0,1050.0,2404.0,0.475945,1111.0,2384.0,23.0,284.0,307.0
592,Pine Grove Ave & Waveland Ave,2046.0,1870.0,3916.0,492.0,694.0,728.0,780.0,1868.0,0.477017,1186.0,1508.0,202.0,52.0,254.0
149,Clark St & Chicago Ave,1507.0,1179.0,2686.0,325.0,518.0,537.0,565.0,1451.0,0.540208,843.0,1102.0,193.0,28.0,221.0
1054,Wentworth Ave & Cermak Rd,887.0,961.0,1848.0,304.0,153.0,412.0,386.0,792.0,0.428571,457.0,798.0,151.0,26.0,177.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,Springfield Ave & 47th St,30.0,14.0,44.0,1.0,9.0,8.0,12.0,26.0,0.590909,10.0,20.0,8.0,4.0,12.0
286,Fort Dearborn Dr & 31st St,396.0,372.0,768.0,117.0,123.0,149.0,155.0,270.0,0.351562,240.0,304.0,6.0,6.0,12.0
289,Francisco Ave & Foster Ave,263.0,289.0,552.0,78.0,83.0,99.0,106.0,237.0,0.429348,161.0,205.0,5.0,7.0,12.0
512,Meade Ave & Addison St,17.0,2.0,19.0,0.0,2.0,2.0,11.0,16.0,0.842105,2.0,13.0,2.0,9.0,11.0


# Получим соотношение подписчиков 

In [18]:
# получаем список первых топ-20 станций
stations = swing_stations["station_name"].unique()
stations = list(stations[:20])

In [40]:
df22 = pd.read_csv(f"Divvy_Trips_2022_full.csv.gz", compression="gzip")

In [41]:
# фильтруем по наличию нужных нам станций
df22 = df22[
    (df22["start_station_name"].isin(stations))
    | (df22["end_station_name"].isin(stations))
]
# берем промежуток времени 2022-02-07 - 2022-02-13
df22 = df22[
    (df22["started_at"] >= "2022-02-07 00:00:00")
    & (df22["started_at"] < "2022-02-14 00:00:00")
]

In [44]:
df22 = df22[["start_station_name", "end_station_name", "member_casual"]]
# группируем типы пользователей по началу и концу поездки
start = df22.groupby("start_station_name").agg(list).reset_index("start_station_name")
end = df22.groupby("end_station_name").agg(list).reset_index("end_station_name")

start = start[["start_station_name", "member_casual"]]
end = end[["end_station_name", "member_casual"]]

In [25]:
start.head()

,start_station_name,member_casual
0,900 W Harrison St,"[member, member, member, member, member, casua..."
1,Aberdeen St & Jackson Blvd,"[member, member, member, member, member]"
2,Aberdeen St & Monroe St,"[member, member, member, member, member, member]"
3,Aberdeen St & Randolph St,"[member, member, member, member, member, member]"
4,Albany Ave & Montrose Ave,"[casual, member, member]"


In [ ]:
end.head()

In [45]:
end.columns = ["start_station_name", "member_casual"]
end = end[end["start_station_name"].isin(stations)]
start = start[start["start_station_name"].isin(stations)]

In [46]:
# конкат объединенной таблицы
df = pd.concat([start, end], axis=0)

In [47]:
# убираем дубли, складываем
df = df.groupby("start_station_name").sum()

In [69]:
# убираем мусорные станции
df = df.reset_index()
df = df[df["start_station_name"].isin(stations)]

In [86]:
st = []
# проходим по датайрейму
for col_name, data in df.iterrows():
    # считаем процент мемберов в листе
    st += [data["member_casual"].count("member") / len(data["member_casual"]) * 100]
mean = sum(st) / len(st)
median = (max(st) + min(st)) / 2
print(mean, median)

82.18815267656898 78.66094221397076
